In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("../butil/tmp").load_data()

In [4]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

3232 

<class 'llama_index.core.schema.Document'>
Doc ID: 72e47d34-085f-45e5-8e94-3bbbd7444fa7
Text: 12 'O' Clock Theatrical release poster Directed by Ram Gopal
Varma Produced by Ram Gopal Varma Starring Mithun Chakraborty Makarand
Deshpande Flora Saini Manav Kaul Krishna Gautam CinematographyAmol
Rathod Music by M. M. Keeravani Production companyCompany Production
Distributed by UFO Moviez Release date 8 January 2021[1] Running time
135 minut...


In [5]:
from llama_index.core import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [6]:
from llama_index.core import VectorStoreIndex
from llama_index.core import ServiceContext
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
service_context = ServiceContext.from_defaults(
    #llm=llm, embed_model=OpenAIEmbedding(embed_batch_size=10)
    llm=llm, embed_model="local:BAAI/bge-small-en-v1.5"
)
index = VectorStoreIndex.from_documents([document],
                                        service_context=service_context)

/var/folders/0j/d0xd373d2tb7zdzcsqrk3mpw0000gn/T/ipykernel_99895/3058746696.py:7: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [30]:
query_engine = index.as_query_engine()

In [31]:
response1 = query_engine.query("What is the plot of Yodha movie?")
print(str(response1))

The plot of the movie "Yodha" revolves around Arun Katyal, an army soldier and member of the Yodha Task Force, his wife Priyamvada, and a flight attendant named Laila Khalid. The story unfolds as Arun's father, Major Surender Katyal, who is the founder of the Yodha Task Force, plays a significant role. The narrative involves various characters like Sameer Khan, Jalal, Tanya Sharma, and others, with different relationships and dynamics. The movie showcases a series of events involving these characters, leading to a climax that involves a hostage situation and a tragic outcome.


In [32]:
response = query_engine.query("Suggest a movie where actor is food delivery rider")
print(str(response))

Tu Hovein Main Hovan


### Evaluation setup using Trulens

In [15]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        print(item)
        eval_questions.append(item)

Tell me a movie that where the main character is food delivery rider
Who's the male actor in Yodha movie?
Which movies grossed grossed  ₹237.44  crore?
Which movies has track Arjan Vailly?
Which movie made second highest on opening day after Jawan?
Which movie has song Dil Jhoom?
Which film was premiered on Netflix from 19 April 2024?


In [10]:
from trulens_eval import Tru
tru = Tru()

tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


In [35]:
from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(query_engine,
                                             app_id="Direct Query Engine")

In [36]:
with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

...

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

In [37]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

In [38]:
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Context Relevance,Groundedness,Answer Relevance_calls,Context Relevance_calls,Groundedness_calls,latency,total_tokens,total_cost
0,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_c0b0fbc206ad57173a7aadbd924779a3,"""Tell me a movie that where the main character...","""Street Dancer 3D""",-,"{""record_id"": ""record_hash_c0b0fbc206ad57173a7...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-06-16T10:45:41.598535"", ""...",2024-06-16T10:45:43.436799,0.0,0.00,1.0,[{'args': {'prompt': 'Tell me a movie that whe...,[{'args': {'prompt': 'Tell me a movie that whe...,"[{'args': {'source': 'Just outside his home, h...",1,2035,0.003055
1,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_7e53e1fa92df1419c59f7fe9d52014cd,"""Who's the male actor in Yodha movie?""","""Sidharth Malhotra""",-,"{""record_id"": ""record_hash_7e53e1fa92df1419c59...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-06-16T10:45:43.747250"", ""...",2024-06-16T10:45:45.800991,0.0,0.00,1.0,[{'args': {'prompt': 'Who's the male actor in ...,[{'args': {'prompt': 'Who's the male actor in ...,[{'args': {'source': 'Y o d h a Theatrical rel...,2,2120,0.003183
2,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_418bea01a5b9bcbd6905ac8a7d19ebca,"""Which movies grossed grossed \u20b9237.44 c...","""Rocky Aur Rani Kii Prem Kahaani grossed \u20b...",-,"{""record_id"": ""record_hash_418bea01a5b9bcbd690...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-06-16T10:45:46.117137"", ""...",2024-06-16T10:45:48.453905,0.2,0.55,1.0,[{'args': {'prompt': 'Which movies grossed gro...,[{'args': {'prompt': 'Which movies grossed gro...,[{'args': {'source': '[248] The dubbe d versio...,2,2102,0.003181
3,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_2265b0e859297d90c9600c8bd7074928,"""Which movies has track Arjan Vailly?""","""The movie \""Baaghi 3\"" has the track \""Arjan ...",-,"{""record_id"": ""record_hash_2265b0e859297d90c96...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-06-16T10:45:48.835098"", ""...",2024-06-16T10:45:51.333866,1.0,0.40,0.0,[{'args': {'prompt': 'Which movies has track A...,[{'args': {'prompt': 'Which movies has track A...,[{'args': {'source': 'Retrieved 19 May 2022. 5...,2,1776,0.002673
4,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_38c0b940604dab95979e9657a9f60faa,"""Which movie made second highest on opening da...","""Pathaan made the second highest on the openin...",-,"{""record_id"": ""record_hash_38c0b940604dab95979...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-06-16T10:45:51.738108"", ""...",2024-06-16T10:45:54.711947,0.0,0.00,0.0,[{'args': {'prompt': 'Which movie made second ...,[{'args': {'prompt': 'Which movie made second ...,[{'args': {'source': '[133] Action Cut Enterta...,2,2083,0.003131


In [39]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://10.191.2.98:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

### Sentence Window Retrieval

In [16]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [19]:
from utils import build_sentence_window_index

In [20]:
sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

In [21]:
from utils import get_sentence_window_query_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index)

In [22]:
tru.reset_database()

tru_recorder_sentence_window = get_prebuilt_trulens_recorder(
    sentence_window_engine,
    app_id = "Sentence Window Query Engine"
)

In [23]:
for question in eval_questions:
    with tru_recorder_sentence_window as recording:
        response = sentence_window_engine.query(question)
        print(question)
        print(str(response))

Tell me a movie that where the main character is food delivery rider
"Kapil Sharma's Zwigato" is a movie where the main character is a food delivery boy.


Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Who's the male actor in Yodha movie?
Sidharth Malhotra


Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

...

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Which movie has song Dil Jhoom?
The movie that has the song "Dil Jhoom" is the one where Jacqueline Fernandez was supposed to pair with Jammwal but was replaced by Nora Fatehi due to creative differences.


Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Which film was premiered on Netflix from 19 April 2024?
The film "Two Plus Two is Love" was premiered on Netflix from 19 April 2024.


In [24]:
tru.get_leaderboard(app_ids=[])

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

,Answer Relevance,Context Relevance,Groundedness,latency,total_cost
app_id,,,,,
Sentence Window Query Engine,0.533333,0.433333,0.333333,9.714286,0.001029


In [25]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://10.191.2.98:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

### Auto-merging retrieval

In [7]:
from utils import build_automerging_index

automerging_index = build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index"
)

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [8]:
from utils import get_automerging_query_engine

automerging_query_engine = get_automerging_query_engine(
    automerging_index,
)

In [13]:
from trulens_eval import Tru
from utils import get_prebuilt_trulens_recorder

tru = Tru()

tru.reset_database()

tru_recorder_automerging = get_prebuilt_trulens_recorder(automerging_query_engine,
                                                         app_id="Automerging Query Engine")

In [16]:
for question in eval_questions:
    with tru_recorder_automerging as recording:
        response = automerging_query_engine.query(question)
        print(question)
        print(response)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Tell me a movie that where the main character is food delivery rider
Indoo Ki Jawani


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

...

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Which movie has song Dil Jhoom?
The movie that features the song "Dil Jhoom" is "Dil".


Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Which film was premiered on Netflix from 19 April 2024?
The film premiered on Netflix from 19 April 2024 is "Article 370."


In [17]:
tru.get_leaderboard(app_ids=[])

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

,Groundedness,Context Relevance,Answer Relevance,latency,total_cost
app_id,,,,,
Automerging Query Engine,0.333333,0.425,0.633333,4.571429,0.000613


In [18]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://10.191.2.98:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]